# Bootstrap Convergence

---

This is a convergence study of the `eval_bootstrap()` utility.

In [ ]:
import grama as gr
import numpy as np
import pandas as pd
import time

## Setup
np.random.seed(101)

# Script settings
mu_true = 0
CON = 0.90

df = pd.DataFrame(
    data=dict(
        X=np.random.normal(size=100, loc=mu_true)
    )
)

## Helper functions
def tran_mean(df):
    return pd.DataFrame(dict(
        mu=[np.mean(df.X)]
    ))

def do_boot(n_boot, n_sub, seed=101):
    return gr.tran_bootstrap(
        df,
        tran=tran_mean,
        n_boot=n_boot,
        n_sub=n_sub,
        seed=seed,
        con=CON
    )

def is_cov(df_res):
    return ((df_res.mu_lo < mu_true) & (mu_true < df_res.mu_up))[0]

In [ ]:
# Loop settings
N_BOOT_ALL = [100, 1000]
N_SUB_ALL = [10, 25, 50]
N_REPL = 40

# Reserve space
BOOL_ALL = np.zeros((len(N_BOOT_ALL), len(N_SUB_ALL), N_REPL))
LO_ALL   = np.zeros((len(N_BOOT_ALL), len(N_SUB_ALL), N_REPL))
UP_ALL   = np.zeros((len(N_BOOT_ALL), len(N_SUB_ALL), N_REPL))

t0 = time.time()
for ind, n_boot in enumerate(N_BOOT_ALL):
    for jnd, n_sub in enumerate(N_SUB_ALL):
        for knd in range(N_REPL):
            df_tmp = do_boot(n_boot, n_sub, seed=101 + knd)
            LO_ALL[ind, jnd, knd] = df_tmp.mu_lo[0]
            UP_ALL[ind, jnd, knd] = df_tmp.mu_up[0]
            BOOL_ALL[ind, jnd, knd] = is_cov(df_tmp)
t1 = time.time()

print("Execution time: {} sec".format(t1 - t0))

In [ ]:
df_results = pd.DataFrame(
    data=np.mean(BOOL_ALL, axis=2),
    columns=N_SUB_ALL
)
df_results["Nb"] = N_BOOT_ALL
gr.tran_gather(df=df_results, key="Ns", value="coverage", cols=N_SUB_ALL)
